### First thing first: data ingestion
Ho scelto logstash per la semplicità d'utilizzo e la configurazione semplice e diretta.
Printo anche su stdout per debug.

```conf
input {
  http {
    id => "tap_http_in"
  }
}

output {
  kafka {
    codec => json
    topic_id => "reddit_posts"
    bootstrap_servers => "kafkaserver:9092"
  }
  stdout { codec => rubydebug }
}
```



 https://www.elastic.co/guide/en/logstash/current/config-examples.html
 

L'interfacciamento con reddit sarà svolto da uno script in python che fa uso della libreria PRAW (Python Reddit API Wrapper) il cui compito sarà quello di fetchare i post in streaming dai subreddits prescelti e inoltrarli a logstash mediante interfaccia HTTP.  
Lo script avrà due modalità d'esecuzione:
1. Streaming: quanto appena descritto, scaricherà i post non appena escono e li inoltrerà periodicamente.
2. SendN: potrà inviare N posts precedentemente archiviati, funzione utile per debug e per demo, si potrà scegliere l'intervallo di tempo tra l'invio di un post e il successivo.
3. Download: scarica N posts e li archivia
 
Lo script sarà conteinerizzato anch'esso e si potranno passare i parametri appena descritti come variabili d'ambiente.
```bash
python3 praw_script.py -mode="send" -n=10 -t=10
# or
python3 praw_script.py -mode="streaming"
# or
python3 praw_script.py -mode="download" -n=100

# Container version
docker run --network tap -e PYTHON_APP=tap10linesConsumer.py --rm -it tap:python

```